In [46]:
'''
Rough and ready code to generate a set of customer transactions. This is assumed to be 
within a not-for-profit direct marketing context in which customers have to be asked to 
make donations, predominately through 3 big campaigns: spring, summer, Christmas.

Customers are created by RecruitmentCampaign.

Customers are then selected a WarmCampaign. There are some rules for selections such that more 
recent donors are likely to be selected.

The decision of a customer to respond to a WarmCampaign is set is partly random but partly down to rules set in 
class ReponseProbabilities.

This was just a quick hack to get some customer transaction data broadly representing a Charity's donor file.


Inputs: None

Outputs: transactions.csv


Note that transactions.csv is the input to the SQL process and is table: dbo.Transactions

'''

from datetime import date
import random
import pandas as pd

#importlib.reload(mymodule)

class File(object):
    def __init__ (self):
        self.lstSupporters = []
        self.latestSupporterID = 0
    
    def addSupporters (self, inLstSupporters):
        for i in inLstSupporters:
            i.setID ( self.latestSupporterID )
            self.latestSupporterID = self.latestSupporterID  + 1

            self.lstSupporters.append (i)
    
    def SelectByCriteria (self, inLTV, inTenureYears):
        return [i for i in self.lstSupporters if i.getTenureYears() <=inTenureYears and i.getLTV() >= inLTV]

    def allSupporters(self):
        return self.lstSupporters
    
    def allTransactionsAsDF (self):
        lstDFs = []
        for s in self.allSupporters():
            lstDFs.append (  s.getTransactionsAsDF () )

        return pd.concat (lstDFs)
        
class Supporter (object):
    def __init__ (self, inRecruitmentChannel, inRecruitmentDate):
        self.recruitmentChannel = inRecruitmentChannel
        self.recruitmentDate    = inRecruitmentDate
        self.transactions = []
        self.tenureDays = 0
        self.recencyDates = 0 
        self.ID = 0

    def getTenureYears(self):
        return int ( self.tenureDays / 365.25 ) 

    def setID (self, inID):
        self.ID = inID
        
    def makeTransaction (self, inDate, inAmount, inType, inIsFirst):
        aTransaction = Transaction ( inDate, inAmount, inType, inIsFirst)
        self.transactions.append ( aTransaction)
        self.tenureDays = (inDate - self.recruitmentDate ).days
        
    def getLTV (self):
        return int ( sum ([i.amount for i in self.transactions ] ) )
                    
    def latestTransction (self):
        pass

    def printSelf (self):
        outStr = 'ID: {}LTV: {},  Tenure: {}, Channel:{}'.format(self.ID
                                                           , self.getLTV()
                                                           , self.getTenureYears()
                                                           , self.recruitmentChannel )
        print ( outStr ) 
      
    def getTransactionsAsDF (self):
        
        dfList = []
        for t in self.transactions:
            dfList.append ( pd.DataFrame( {'ID':self.ID
                                       ,'Amount':t.amount
                                       ,'TransactionDate': t.transactionDate
                                       , 'Is_First': t.isFirst}  )   )

        return pd.concat (dfList )
            
    
class Transaction (object):
    def __init__ (self, inDate, inAmount, inType, inIsFirst ):
        self.transactionDate = inDate,
        self.amount = inAmount
        self.type            = inType # Cash, RG
        self.isFirst         = inIsFirst

class Transactions (object):
    def __init__ (self):
        self._transactionList = []

class Campaign(object):
    def processTransactions (self, supporterSelection):
            pass

class RecruitmentCampaign(object):
    def __init__ (self, inCampaignDate, inChannel ):
        self.campaignDate = inCampaignDate
        self.channel      = inChannel
        
    def getSupporters (self, inNumSupporters):
        lstSupporters = []
        for i in range(1,inNumSupporters+1):
            s = Supporter(self.channel, self.campaignDate )
            s.makeTransaction (self.campaignDate, abs ( random.gauss (100, 200) ),  'CASH', 'YES' )
            lstSupporters.append (s) 
        return lstSupporters

class WarmCampaign(object):

    def __init__ (self, inCampaignDate, inFile):
        self.campaignDate = inCampaignDate
        self.file = inFile        
        self.selectionLst = []
        self.donorList = []

    def runSupporterSelection (self, inMaxTenureYrs, inMinLTV ):
        self.selectionLst = self.file.SelectByCriteria (inMaxTenureYrs, inMinLTV)

    def runDonorList (self):

        self.donorList = []
        
        for s in self.selectionLst:

            prob = ReponseProbabilities.getProbability (s.recruitmentChannel, self.campaignDate.month, s.getTenureYears(), s.getLTV()  )  
            if prob >= 0.5:
                self.donorList.append ( s)

    def processPayments (self):
        for s in self.donorList:
            s.makeTransaction (self.campaignDate, abs ( random.gauss (50, 50) ),  'CASH', 'NO' )

    def getResponseRate (self):
        if  len (self.selectionLst) == 0:
            return 0
        return len (self.donorList) / len (self.selectionLst)
    
    def printSelf (self):
        outStr = 'WC: Year:{}, Month:{}, Selected: {}, Donated:{}'.format (self.campaignDate.year
                                                                      , self.campaignDate.month
                                                                      , len (self.selectionLst)
                                                                       ,len (self.donorList)  )
        print (outStr)

class ReponseProbabilities(object):
    
    def getProbability ( inRecruitmentChannel, inMonth, inTenureYears, inLTV ):
        
        channelProb = 1.0; monthProb = 1.0; tenureProb = 1.0; LTVprob = 1.0;
        
        if inRecruitmentChannel == 'MAIL':
            channelProb = 0.9
        else:
            channelProb = 0.7
        
        if inMonth ==  4:
            monthProb = 0.6
        elif inMonth ==  7:
            monthProb = 0.8        
        elif inMonth ==  12:
            monthProb = 0.9        
 
        if int(inTenureYears) <= 1.0:
            tenureProb = 0.9
        if int(inTenureYears) <= 2.0:
            tenureProb = 0.7
        elif inTenureYears <=  4:
            tenureProb = 0.4      
        else:
            tenureProb = 0.1        
           
        if int(inLTV) >= 400:
            LTVprob = 0.9
        elif int(inLTV) >=  50:
            LTVprob = 0.7
        else:
            LTVprob = 0.4
        
        return tenureProb * channelProb * monthProb * LTVprob * random.gauss (0.4, 0.4) 


class CampaignScheduler(object):
    def processCampaigns (inFile):

        lstChannels = ['WEB','MAIL']
        lstMonths   =   [1,2,3,4,5,6,7,8,9,10,11,12]                  
        yrRange     = [yr for yr in range (1990,2021+1)]
        for yr in yrRange:
            for ch in lstChannels:
                for mnth in lstMonths:
                    rcDate = date ( yr,mnth, 1)

                    rc = RecruitmentCampaign (rcDate, ch )

                    numRecruits = 0
                    if mnth == 12:
                        numRecruits = 200
                    elif mnth == 7:
                        numRecruits = 100
                    else:
                        numRecruits = 50

                    numRecruits = numRecruits + ((2020 - yr)*4 )
                    

                    
                    if yr == 2019:
                        numRecruits += 200
                    if yr == 2020:
                        numRecruits += 400                    
                    if yr == 2021:
                        numRecruits += 100                                        
                    
                    if mnth in ( 4,7,12):
                        inFile.addSupporters ( rc.getSupporters (numRecruits) )
                        print ('Added {}'.format ( numRecruits))
                    
                    wc = WarmCampaign (rcDate, inFile )
                    wc.runSupporterSelection ( 4, 10 )
                    wc.runDonorList ( )
                    wc.processPayments ()
                    wc.printSelf()
                    
                    print ('Warm Campaign Year {},Month :{}, Campaign:{} had RR of: {}'.format (rcDate.year
                                                                                   ,rcDate.month
                                                                                   ,ch
                                                                                   ,wc.getResponseRate() ) )

f = File ()                    
CampaignScheduler.processCampaigns(f)

df = f.allTransactionsAsDF () 
df.to_csv ('transactions.csv')




WC: Year:1990, Month:1, Selected: 0, Donated:0
Warm Campaign Year 1990,Month :1, Campaign:WEB had RR of: 0
WC: Year:1990, Month:2, Selected: 0, Donated:0
Warm Campaign Year 1990,Month :2, Campaign:WEB had RR of: 0
WC: Year:1990, Month:3, Selected: 0, Donated:0
Warm Campaign Year 1990,Month :3, Campaign:WEB had RR of: 0
Added 170
WC: Year:1990, Month:4, Selected: 165, Donated:0
Warm Campaign Year 1990,Month :4, Campaign:WEB had RR of: 0.0
WC: Year:1990, Month:5, Selected: 165, Donated:0
Warm Campaign Year 1990,Month :5, Campaign:WEB had RR of: 0.0
WC: Year:1990, Month:6, Selected: 165, Donated:0
Warm Campaign Year 1990,Month :6, Campaign:WEB had RR of: 0.0
Added 220
WC: Year:1990, Month:7, Selected: 382, Donated:0
Warm Campaign Year 1990,Month :7, Campaign:WEB had RR of: 0.0
WC: Year:1990, Month:8, Selected: 382, Donated:2
Warm Campaign Year 1990,Month :8, Campaign:WEB had RR of: 0.005235602094240838
WC: Year:1990, Month:9, Selected: 382, Donated:3
Warm Campaign Year 1990,Month :9, Camp

WC: Year:1992, Month:5, Selected: 3619, Donated:79
Warm Campaign Year 1992,Month :5, Campaign:MAIL had RR of: 0.02182923459519204
WC: Year:1992, Month:6, Selected: 3619, Donated:70
Warm Campaign Year 1992,Month :6, Campaign:MAIL had RR of: 0.019342359767891684
Added 212
WC: Year:1992, Month:7, Selected: 3830, Donated:15
Warm Campaign Year 1992,Month :7, Campaign:MAIL had RR of: 0.0039164490861618795
WC: Year:1992, Month:8, Selected: 3830, Donated:83
Warm Campaign Year 1992,Month :8, Campaign:MAIL had RR of: 0.0216710182767624
WC: Year:1992, Month:9, Selected: 3830, Donated:75
Warm Campaign Year 1992,Month :9, Campaign:MAIL had RR of: 0.0195822454308094
WC: Year:1992, Month:10, Selected: 3830, Donated:63
Warm Campaign Year 1992,Month :10, Campaign:MAIL had RR of: 0.016449086161879897
WC: Year:1992, Month:11, Selected: 3830, Donated:79
Warm Campaign Year 1992,Month :11, Campaign:MAIL had RR of: 0.0206266318537859
Added 312
WC: Year:1992, Month:12, Selected: 4139, Donated:43
Warm Campaign

WC: Year:1995, Month:7, Selected: 7120, Donated:21
Warm Campaign Year 1995,Month :7, Campaign:WEB had RR of: 0.002949438202247191
WC: Year:1995, Month:8, Selected: 7120, Donated:120
Warm Campaign Year 1995,Month :8, Campaign:WEB had RR of: 0.016853932584269662
WC: Year:1995, Month:9, Selected: 7120, Donated:115
Warm Campaign Year 1995,Month :9, Campaign:WEB had RR of: 0.016151685393258428
WC: Year:1995, Month:10, Selected: 7120, Donated:103
Warm Campaign Year 1995,Month :10, Campaign:WEB had RR of: 0.01446629213483146
WC: Year:1995, Month:11, Selected: 7120, Donated:111
Warm Campaign Year 1995,Month :11, Campaign:WEB had RR of: 0.015589887640449438
Added 300
WC: Year:1995, Month:12, Selected: 7414, Donated:58
Warm Campaign Year 1995,Month :12, Campaign:WEB had RR of: 0.007823037496628
WC: Year:1995, Month:1, Selected: 7414, Donated:125
Warm Campaign Year 1995,Month :1, Campaign:MAIL had RR of: 0.016859994604801726
WC: Year:1995, Month:2, Selected: 7414, Donated:121
Warm Campaign Year 1

WC: Year:1997, Month:9, Selected: 10267, Donated:122
Warm Campaign Year 1997,Month :9, Campaign:MAIL had RR of: 0.011882731080159735
WC: Year:1997, Month:10, Selected: 10267, Donated:113
Warm Campaign Year 1997,Month :10, Campaign:MAIL had RR of: 0.011006136164410247
WC: Year:1997, Month:11, Selected: 10267, Donated:111
Warm Campaign Year 1997,Month :11, Campaign:MAIL had RR of: 0.010811337294243693
Added 292
WC: Year:1997, Month:12, Selected: 10556, Donated:71
Warm Campaign Year 1997,Month :12, Campaign:MAIL had RR of: 0.006726032588101553
WC: Year:1998, Month:1, Selected: 10556, Donated:132
Warm Campaign Year 1998,Month :1, Campaign:WEB had RR of: 0.012504736642667677
WC: Year:1998, Month:2, Selected: 10556, Donated:131
Warm Campaign Year 1998,Month :2, Campaign:WEB had RR of: 0.012410003789314134
WC: Year:1998, Month:3, Selected: 10556, Donated:127
Warm Campaign Year 1998,Month :3, Campaign:WEB had RR of: 0.012031072375899962
Added 138
WC: Year:1998, Month:4, Selected: 10689, Donate

WC: Year:2000, Month:10, Selected: 13251, Donated:127
Warm Campaign Year 2000,Month :10, Campaign:WEB had RR of: 0.009584182325862198
WC: Year:2000, Month:11, Selected: 13251, Donated:125
Warm Campaign Year 2000,Month :11, Campaign:WEB had RR of: 0.00943325032073051
Added 280
WC: Year:2000, Month:12, Selected: 13527, Donated:58
Warm Campaign Year 2000,Month :12, Campaign:WEB had RR of: 0.004287720854587122
WC: Year:2000, Month:1, Selected: 13527, Donated:119
Warm Campaign Year 2000,Month :1, Campaign:MAIL had RR of: 0.008797220374066682
WC: Year:2000, Month:2, Selected: 13527, Donated:105
Warm Campaign Year 2000,Month :2, Campaign:MAIL had RR of: 0.007762253271235307
WC: Year:2000, Month:3, Selected: 13527, Donated:102
Warm Campaign Year 2000,Month :3, Campaign:MAIL had RR of: 0.00754047460634287
Added 130
WC: Year:2000, Month:4, Selected: 13655, Donated:3
Warm Campaign Year 2000,Month :4, Campaign:MAIL had RR of: 0.00021969974368363237
WC: Year:2000, Month:5, Selected: 13655, Donated:

...

WC: Year:2010, Month:2, Selected: 22946, Donated:116
Warm Campaign Year 2010,Month :2, Campaign:MAIL had RR of: 0.005055347337226532
WC: Year:2010, Month:3, Selected: 22935, Donated:106
Warm Campaign Year 2010,Month :3, Campaign:MAIL had RR of: 0.004621757139742751
Added 90
WC: Year:2010, Month:4, Selected: 23017, Donated:2
Warm Campaign Year 2010,Month :4, Campaign:MAIL had RR of: 8.689229699787114e-05
WC: Year:2010, Month:5, Selected: 23017, Donated:115
Warm Campaign Year 2010,Month :5, Campaign:MAIL had RR of: 0.004996307077377591
WC: Year:2010, Month:6, Selected: 23007, Donated:99
Warm Campaign Year 2010,Month :6, Campaign:MAIL had RR of: 0.004303038205763463
Added 140
WC: Year:2010, Month:7, Selected: 23134, Donated:22
Warm Campaign Year 2010,Month :7, Campaign:MAIL had RR of: 0.0009509812397337252
WC: Year:2010, Month:8, Selected: 23133, Donated:102
Warm Campaign Year 2010,Month :8, Campaign:MAIL had RR of: 0.0044092854363895735
WC: Year:2010, Month:9, Selected: 23124, Donated:10

WC: Year:2013, Month:3, Selected: 24662, Donated:120
Warm Campaign Year 2013,Month :3, Campaign:WEB had RR of: 0.0048657854188630285
Added 78
WC: Year:2013, Month:4, Selected: 24723, Donated:1
Warm Campaign Year 2013,Month :4, Campaign:WEB had RR of: 4.0448165675686606e-05
WC: Year:2013, Month:5, Selected: 24723, Donated:120
Warm Campaign Year 2013,Month :5, Campaign:WEB had RR of: 0.004853779881082393
WC: Year:2013, Month:6, Selected: 24711, Donated:116
Warm Campaign Year 2013,Month :6, Campaign:WEB had RR of: 0.0046942657116264015
Added 128
WC: Year:2013, Month:7, Selected: 24823, Donated:21
Warm Campaign Year 2013,Month :7, Campaign:WEB had RR of: 0.0008459896064134069
WC: Year:2013, Month:8, Selected: 24822, Donated:126
Warm Campaign Year 2013,Month :8, Campaign:WEB had RR of: 0.005076142131979695
WC: Year:2013, Month:9, Selected: 24809, Donated:94
Warm Campaign Year 2013,Month :9, Campaign:WEB had RR of: 0.0037889475593534602
WC: Year:2013, Month:10, Selected: 24797, Donated:120
W

WC: Year:2015, Month:4, Selected: 26250, Donated:1
Warm Campaign Year 2015,Month :4, Campaign:MAIL had RR of: 3.809523809523809e-05
WC: Year:2015, Month:5, Selected: 26250, Donated:106
Warm Campaign Year 2015,Month :5, Campaign:MAIL had RR of: 0.004038095238095238
WC: Year:2015, Month:6, Selected: 26237, Donated:104
Warm Campaign Year 2015,Month :6, Campaign:MAIL had RR of: 0.00396386782025384
Added 120
WC: Year:2015, Month:7, Selected: 26336, Donated:16
Warm Campaign Year 2015,Month :7, Campaign:MAIL had RR of: 0.0006075334143377885
WC: Year:2015, Month:8, Selected: 26336, Donated:107
Warm Campaign Year 2015,Month :8, Campaign:MAIL had RR of: 0.004062879708383961
WC: Year:2015, Month:9, Selected: 26321, Donated:109
Warm Campaign Year 2015,Month :9, Campaign:MAIL had RR of: 0.004141180046350823
WC: Year:2015, Month:10, Selected: 26307, Donated:110
Warm Campaign Year 2015,Month :10, Campaign:MAIL had RR of: 0.004181396586459878
WC: Year:2015, Month:11, Selected: 26296, Donated:112
Warm 

WC: Year:2018, Month:6, Selected: 27559, Donated:93
Warm Campaign Year 2018,Month :6, Campaign:WEB had RR of: 0.003374578177727784
Added 108
WC: Year:2018, Month:7, Selected: 27648, Donated:14
Warm Campaign Year 2018,Month :7, Campaign:WEB had RR of: 0.0005063657407407407
WC: Year:2018, Month:8, Selected: 27648, Donated:88
Warm Campaign Year 2018,Month :8, Campaign:WEB had RR of: 0.00318287037037037
WC: Year:2018, Month:9, Selected: 27635, Donated:86
Warm Campaign Year 2018,Month :9, Campaign:WEB had RR of: 0.0031119956576804776
WC: Year:2018, Month:10, Selected: 27621, Donated:104
Warm Campaign Year 2018,Month :10, Campaign:WEB had RR of: 0.003765251077079034
WC: Year:2018, Month:11, Selected: 27607, Donated:96
Warm Campaign Year 2018,Month :11, Campaign:WEB had RR of: 0.003477378925634803
Added 208
WC: Year:2018, Month:12, Selected: 27801, Donated:43
Warm Campaign Year 2018,Month :12, Campaign:WEB had RR of: 0.0015467069529873027
WC: Year:2018, Month:1, Selected: 27800, Donated:87
Wa

WC: Year:2020, Month:7, Selected: 31954, Donated:27
Warm Campaign Year 2020,Month :7, Campaign:MAIL had RR of: 0.0008449646366652063
WC: Year:2020, Month:8, Selected: 31954, Donated:159
Warm Campaign Year 2020,Month :8, Campaign:MAIL had RR of: 0.0049759028603617705
WC: Year:2020, Month:9, Selected: 31943, Donated:153
Warm Campaign Year 2020,Month :9, Campaign:MAIL had RR of: 0.004789781798829164
WC: Year:2020, Month:10, Selected: 31919, Donated:138
Warm Campaign Year 2020,Month :10, Campaign:MAIL had RR of: 0.004323443716908425
WC: Year:2020, Month:11, Selected: 31909, Donated:161
Warm Campaign Year 2020,Month :11, Campaign:MAIL had RR of: 0.005045598420508321
Added 600
WC: Year:2020, Month:12, Selected: 32482, Donated:90
Warm Campaign Year 2020,Month :12, Campaign:MAIL had RR of: 0.002770765346961394
WC: Year:2021, Month:1, Selected: 32479, Donated:172
Warm Campaign Year 2021,Month :1, Campaign:WEB had RR of: 0.005295729548323532
WC: Year:2021, Month:2, Selected: 32459, Donated:149
W